In [ ]:
!pip install colorednoise > /dev/null

## About

このノートブックでは音データに対するData Augmentationを紹介します。画像同様、音のデータもData Augmentationは汎化性能を担保する上で大きな役割を果たしています。

In this notebook, I will introduce some basic Data Augmentation methods for audio. Similar in computer vision, Data Augmentation is quite effective for audio as well to make generalized model.

Maybe you'll find some odd translation since I translated this from Japanese and in some part I just used the result of deepL...

In [ ]:
import librosa
import librosa.display

import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path
from IPython.display import Audio

音のデータに適用できるData Augmentationは画像のものとは大きく異なります。音のデータには大きく分けて以下の二種類のData Augmentationが存在します。

1. 元の音データ(1次元の状態)に対して適用されるData Augmentaion (Data Augmentation for waveform)
2. スペクトログラムやメルスペクトログラムに対して適用されるData Augmentation (Data Augmentation for spectrogram)

元の音データに対して適用されるData Augmentationは音の聞こえ方自体を変化させるものが多く画像のものとは大きく異なります。

一方スペクトログラムに対して適用されるものは画像に対するData Augmentationに似ていますが、一つ大きく異なる点として、スペクトログラムには明確に軸があるという点を無視してData Augmentationをかけることには意味がありません。軸というのは、スペクトログラムのx軸は時間、y軸は周波数の軸であるということでここは明確に自然画像とは異なります。従って単純にFlipしたり回転をかけることは意味がないどころか有害です。

Data Augmentations for audio is quite different from those for images. There are two types of augmentations:

1. Augmentations for waveform
2. Augmentations for spectrogram/melspectrogram

Augmentation for waveform is applied to raw 1D signal, and changes how it sounds like, so we can check how it alters the signal by listening it.
On the other hand, augmentation for spectrogram is something similar to image augmentation. However, spectrogram has some big differences from natural image.
One of the biggest difference is that it has axis - time axis and frequency axis. Applying augmentations that ignores this axis (Flip, Rotation, etc...) is nonsense.

## Data Loading




In [ ]:
DATA_DIR = Path("../input/rfcx-species-audio-detection/train")

In [ ]:
flacfiles = list(DATA_DIR.glob("*.flac"))
y, sr = librosa.load(flacfiles[0], duration=10)
y, sr

音のデータを読み込むことができました。今回はこのデータに様々なData Augmentationをかけてその結果を実際に聞いたり見たりしながら各Data Augmentationの効果を確認していきます。

まずは元の音データを確認してみましょう。

Now we've loaded audio data. In this notebook, we'll apply several Data Augmentations and check the result by listening or visually watching it.

In [ ]:
Audio(y, rate=sr)

In [ ]:
librosa.display.waveplot(y, sr=sr);

In [ ]:
melspec = librosa.power_to_db(librosa.feature.melspectrogram(y, sr=sr, n_mels=128))
librosa.display.specshow(melspec, sr=sr, x_axis="time", y_axis="mel")
plt.colorbar();

## 音データに対するData Augmentation (Data Augmentation for waveform)

まずは、音の一次元データに対するData Augmentationを紹介します。ここでは実際に実装しながら以下のaugmentationを紹介します。

1. AddGaussianNoise
2. GaussianNoiseSNR
3. PinkNoiseSNR
4. PitchShift
5. TimeStretch
6. TimeShift
7. VolumeControl

なお、実装は[albumentations](https://github.com/albumentations-team/albumentations)を参考にしています。albumentationsのインターフェースを継承して作成をすることもできるのですが若干手間が多いため自前実装で済ませます。

Here, I'll introduce some Data Augmentation methods for raw waveform.

1. AddGaussianNoise
2. GaussianNoiseSNR
3. PinkNoiseSNR
4. PitchShift
5. TimeStretch
6. TimeShift
7. VolumeControl

I imitated the implementation of [albumentations](https://github.com/albumentations-team/albumentations).

In [ ]:
class AudioTransform:
    def __init__(self, always_apply=False, p=0.5):
        self.always_apply = always_apply
        self.p = p

    def __call__(self, y: np.ndarray):
        if self.always_apply:
            return self.apply(y)
        else:
            if np.random.rand() < self.p:
                return self.apply(y)
            else:
                return y

    def apply(self, y: np.ndarray):
        raise NotImplementedError


class Compose:
    def __init__(self, transforms: list):
        self.transforms = transforms

    def __call__(self, y: np.ndarray):
        for trns in self.transforms:
            y = trns(y)
        return y


class OneOf:
    def __init__(self, transforms: list):
        self.transforms = transforms

    def __call__(self, y: np.ndarray):
        n_trns = len(self.transforms)
        trns_idx = np.random.choice(n_trns)
        trns = self.transforms[trns_idx]
        return trns(y)

### AddGaussianNoise

正規分布に従うノイズ、いわゆるホワイトノイズを音に加えます。ノイズの振幅はランダムです。

参考・出典: [Data Augmentation for Audio](https://medium.com/@makcedward/data-augmentation-for-audio-76912b01fdf6)の`NoiseInjection`より

Add noise that follows normal distribution (a.k.a whitenoise). The amplitude of noise is randomly decided.

Reference: [Data Augmentation for Audio](https://medium.com/@makcedward/data-augmentation-for-audio-76912b01fdf6)

In [ ]:
class AddGaussianNoise(AudioTransform):
    def __init__(self, always_apply=False, p=0.5, max_noise_amplitude=0.5, **kwargs):
        super().__init__(always_apply, p)

        self.noise_amplitude = (0.0, max_noise_amplitude)

    def apply(self, y: np.ndarray, **params):
        noise_amplitude = np.random.uniform(*self.noise_amplitude)
        noise = np.random.randn(len(y))
        augmented = (y + noise * noise_amplitude).astype(y.dtype)
        return augmented

In [ ]:
transform = AddGaussianNoise(always_apply=True, max_noise_amplitude=0.05)
y_gaussian_added = transform(y)
Audio(y_gaussian_added, rate=sr)

In [ ]:
librosa.display.waveplot(y_gaussian_added, sr=sr);

In [ ]:
melspec = librosa.power_to_db(librosa.feature.melspectrogram(y_gaussian_added, sr=sr, n_mels=128))
librosa.display.specshow(melspec, sr=sr, x_axis="time", y_axis="mel")
plt.colorbar();

スペクトログラムを見るとわかりやすいのですが、全体に散らしたようにノイズが加わっていることがわかります。Gaussian Noiseを加えることでノイズが比較的少ないデータで学習して、ノイズが多い環境の音を処理しなければいけない場合などに汎化性能をあげることができます。

### GaussianNoiseSNR

上で紹介したData Augmentationの問題点はノイズの強さ(amplitude)を指定してしまうと、元の信号が微弱なときに雑音に覆い隠されてしまうことがありうる点です。

これを防ぐために元の音の中の信号の振幅を元に適切な雑音レベルを適応的に設定できるようにしたほうが使いやすいです。

信号の大きさと雑音の大きさの比を表したものをSignal-to-Noise Ratio(SNR)と呼びます。信号の大きさ、といった場合には振幅のことをさすことが多いのですが多くの場合SNRは実際の振幅の比に対数を取ったものとして表現され、以下の式で計算されます。

$$
SNR = 20\log_{10}\frac{A_{signal}}{A_{noise}}
$$

この量は大きければ大きいほど信号が強い、すなわち音が聞こえやすいことを表す量で単位はdB(デシベル)で表現されます。0dBで信号の強さと雑音の強さが釣り合っている状態で、負の場合には雑音の方が強い状態、正の場合には信号の方が強い状態です。

また、信号音の強さの推定法はいくつかあるかと思いますが、今回はクリップ内の振幅の絶対値の最大値を信号の振幅として扱います。

参考・出典: [任意のSignal-to-Noise比の音声波形をPythonで作ろう！](https://engineering.linecorp.com/ja/blog/voice-waveform-arbitrary-signal-to-noise-ratio-python/)より

The problem with Data Augmentation introduced above is that if you specify an amplitude of noise, it can be masked by noise when the original signal is weak.

To prevent this, it is easier to adaptively set an appropriate noise level based on the amplitude of the signal in the original sound.

The ratio of the signal-to-noise level is called Signal-to-Noise Ratio (SNR). The signal-to-noise ratio (SNR) is expressed as the ratio of the actual amplitude to the logarithm of the signal's amplitude and is calculated by the following formula.

$$
SNR = 20\log_{10}\frac{A_{signal}}{A_{noise}}
$$

The larger this amount is, the stronger the signal, or the more audible the sound is, and it is expressed in dB (decibel), where 0dB means that the strength of the signal is balanced with the strength of the noise, when it is negative, the noise is stronger, and when it is positive, the signal is stronger.

There may be several ways to estimate the strength of the signal sound, but in this case we will treat the absolute maximum of the amplitude in the clip as the amplitude of the signal.

Reference: [任意のSignal-to-Noise比の音声波形をPythonで作ろう！](https://engineering.linecorp.com/ja/blog/voice-waveform-arbitrary-signal-to-noise-ratio-python/) (Japanese)

In [ ]:
class GaussianNoiseSNR(AudioTransform):
    def __init__(self, always_apply=False, p=0.5, min_snr=5.0, max_snr=20.0, **kwargs):
        super().__init__(always_apply, p)

        self.min_snr = min_snr
        self.max_snr = max_snr

    def apply(self, y: np.ndarray, **params):
        snr = np.random.uniform(self.min_snr, self.max_snr)
        a_signal = np.sqrt(y ** 2).max()
        a_noise = a_signal / (10 ** (snr / 20))

        white_noise = np.random.randn(len(y))
        a_white = np.sqrt(white_noise ** 2).max()
        augmented = (y + white_noise * 1 / a_white * a_noise).astype(y.dtype)
        return augmented

In [ ]:
transform = GaussianNoiseSNR(always_apply=True, min_snr=5, max_snr=20)
y_gaussian_snr = transform(y)
Audio(y_gaussian_snr, rate=sr)

In [ ]:
librosa.display.waveplot(y_gaussian_snr, sr=sr);

In [ ]:
melspec = librosa.power_to_db(librosa.feature.melspectrogram(y_gaussian_snr, sr=sr, n_mels=128))
librosa.display.specshow(melspec, sr=sr, x_axis="time", y_axis="mel")
plt.colorbar();

### PinkNoiseSNR

Gaussian Noiseはいわゆる白色雑音(white noise)で全周波数帯にノイズをかけるものでした。ここで紹介するPink Noiseは低周波数帯から低周波数帯にかけて徐々にノイズの強さが減少するようなノイズのことをさします。自然界に存在するノイズはこのようなノイズであるとされます。

なお、白色雑音以外のノイズはcolored noiseと呼ばれ、他にはブラウンノイズ、ブルーノイズなど様々なノイズが提案されています。

Pink Noiseを発生させるために`colorednoise`というライブラリを用いるのですがその名前の由来は上記のようなものになります。

なお、白色雑音の時は最初に強さを直接指定するような実装を紹介しましたが、今回はいきなり強さをSNRベースで適応的に決定できる実装を紹介します。

参考・出典: [Wikipedia カラードノイズ](https://ja.wikipedia.org/wiki/%E3%82%AB%E3%83%A9%E3%83%BC%E3%83%89%E3%83%8E%E3%82%A4%E3%82%BA#:~:text=%E3%82%AB%E3%83%A9%E3%83%BC%E3%83%89%E3%83%8E%E3%82%A4%E3%82%BA%EF%BC%88%E8%8B%B1%3A%20colors%20of,%E3%81%9D%E3%81%AE%E7%89%B9%E6%80%A7%E3%82%82%E5%A4%A7%E3%81%8D%E3%81%8F%E7%95%B0%E3%81%AA%E3%82%8B%E3%80%82), [`colorednoise`](https://github.com/felixpatzelt/colorednoise)

Gaussian Noise is a so-called white noise, which is a noise over the whole frequency range. Pink noise, which we introduce here, is noise with a gradual decrease in noise intensity from low frequency to low frequency bands. The noise in the natural world is said to be such noise.

The noise other than white noise is called "colored noise", and various noises such as brown noise and blue noise have been proposed.

The `colorednoise` library is used to generate pink noise, and its name comes from the above.

In the previous article, we introduced an implementation of white noise that directly specifies the intensity of the noise.

Reference: [Wikipedia Colors of noise](https://en.wikipedia.org/wiki/Colors_of_noise), [`colorednoise`](https://github.com/felixpatzelt/colorednoise)

In [ ]:
import colorednoise as cn


class PinkNoiseSNR(AudioTransform):
    def __init__(self, always_apply=False, p=0.5, min_snr=5.0, max_snr=20.0, **kwargs):
        super().__init__(always_apply, p)

        self.min_snr = min_snr
        self.max_snr = max_snr

    def apply(self, y: np.ndarray, **params):
        snr = np.random.uniform(self.min_snr, self.max_snr)
        a_signal = np.sqrt(y ** 2).max()
        a_noise = a_signal / (10 ** (snr / 20))

        pink_noise = cn.powerlaw_psd_gaussian(1, len(y))
        a_pink = np.sqrt(pink_noise ** 2).max()
        augmented = (y + pink_noise * 1 / a_pink * a_noise).astype(y.dtype)
        return augmented

In [ ]:
transform = PinkNoiseSNR(always_apply=True, min_snr=5.0, max_snr=20.0)
y_pink_noise = transform(y)
Audio(y_pink_noise, rate=sr)

In [ ]:
librosa.display.waveplot(y_pink_noise, sr=sr);

In [ ]:
melspec = librosa.power_to_db(librosa.feature.melspectrogram(y_pink_noise, sr=sr, n_mels=128))
librosa.display.specshow(melspec, sr=sr, x_axis="time", y_axis="mel")
plt.colorbar();

### PitchShift

PitchShiftは音のピッチ(高低)に関する調整を施すData Augmentationで、効果として聞こえる音が高く/低くなります。メルスペクトログラム上では、パターンのある周波数帯が上または下にズレます。

PitchShiftはリサンプリングを行うため今まで紹介したData Augmentationと比べると時間がかかるほか、ピッチを変えすぎると音割れを起こしてしまうこともあるため注意が必要です。

参考・出典: [librosa.effects.pitch_shift](http://man.hubwiz.com/docset/LibROSA.docset/Contents/Resources/Documents/generated/librosa.effects.pitch_shift.html)

PitchShift is a data augmentation that adjusts the pitch of the sound (high and low), making the sound heard as an effect higher/lower. On the Meru spectrogram, certain frequency bands in the pattern will be shifted up or down.

PitchShift takes more time than the previously introduced Data Augmentation because of resampling, and you should be careful not to change the pitch too much because it may cause the sound to crack.

Reference: [librosa.effects.pitch_shift](http://man.hubwiz.com/docset/LibROSA.docset/Contents/Resources/Documents/generated/librosa.effects.pitch_shift.html)

In [ ]:
class PitchShift(AudioTransform):
    def __init__(self, always_apply=False, p=0.5, max_steps=5, sr=32000):
        super().__init__(always_apply, p)

        self.max_steps = max_steps
        self.sr = sr

    def apply(self, y: np.ndarray, **params):
        n_steps = np.random.randint(-self.max_steps, self.max_steps)
        augmented = librosa.effects.pitch_shift(y, sr=self.sr, n_steps=n_steps)
        return augmented

In [ ]:
transform = PitchShift(always_apply=True, max_steps=5, sr=sr)
y_pitch_shift = transform(y)
Audio(y_pitch_shift, rate=sr)

In [ ]:
librosa.display.waveplot(y_pitch_shift, sr=sr);

In [ ]:
melspec = librosa.power_to_db(librosa.feature.melspectrogram(y_pitch_shift, sr=sr, n_mels=128))
librosa.display.specshow(melspec, sr=sr, x_axis="time", y_axis="mel")
plt.colorbar();

### TimeStretch

TimeStretchは元の音を時間的に引き延ばしたり圧縮したりします。結果として音のスピードが速くなったり遅くなったりします。

TimeStretchも時間がかかるData Augmentationです。

参考・出典: [librosa.effects.time_stretch](http://man.hubwiz.com/docset/LibROSA.docset/Contents/Resources/Documents/generated/librosa.effects.time_stretch.html)

TimeStretch stretches and compresses the original sound in time. As a result, the speed of the sound may be increased or decreased.

TimeStretch is another time-consuming form of data augmentation.

Reference: [librosa.effects.time_stretch](http://man.hubwiz.com/docset/LibROSA.docset/Contents/Resources/Documents/generated/librosa.effects.time_stretch.html)

In [ ]:
class TimeStretch(AudioTransform):
    def __init__(self, always_apply=False, p=0.5, max_rate=1.2):
        super().__init__(always_apply, p)

        self.max_rate = max_rate

    def apply(self, y: np.ndarray, **params):
        rate = np.random.uniform(0, self.max_rate)
        augmented = librosa.effects.time_stretch(y, rate)
        return augmented

In [ ]:
transform = TimeStretch(always_apply=True, max_rate=2.0)
y_time_stretch = transform(y)
Audio(y_time_stretch, rate=sr)

In [ ]:
librosa.display.waveplot(y_time_stretch, sr=sr);

In [ ]:
melspec = librosa.power_to_db(librosa.feature.melspectrogram(y_time_stretch, sr=sr, n_mels=128))
librosa.display.specshow(melspec, sr=sr, x_axis="time", y_axis="mel")
plt.colorbar();

### TimeShift

TimeShiftは時間的に音イベントをずらすような操作です。ズラした結果、元の音クリップの長さからはみ出した部分の扱いに関しては、前(または後ろ)に持っていってくっつける、無視して捨ててしまう、などのやり方があります。

参考・出典: [Data Augmentation for Audio](https://medium.com/@makcedward/data-augmentation-for-audio-76912b01fdf6)

TimeShift is such an operation that shifts a sound event in time. As for dealing with the part of the sound clip that goes out of the original length as a result of shifting, you can bring it forward (or backward) and stick it to the front (or backward), or ignore it and throw it away.

Reference: [Data Augmentation for Audio](https://medium.com/@makcedward/data-augmentation-for-audio-76912b01fdf6)

In [ ]:
class TimeShift(AudioTransform):
    def __init__(self, always_apply=False, p=0.5, max_shift_second=2, sr=32000, padding_mode="replace"):
        super().__init__(always_apply, p)
    
        assert padding_mode in ["replace", "zero"], "`padding_mode` must be either 'replace' or 'zero'"
        self.max_shift_second = max_shift_second
        self.sr = sr
        self.padding_mode = padding_mode

    def apply(self, y: np.ndarray, **params):
        shift = np.random.randint(-self.sr * self.max_shift_second, self.sr * self.max_shift_second)
        augmented = np.roll(y, shift)
        if self.padding_mode == "zero":
            if shift > 0:
                augmented[:shift] = 0
            else:
                augmented[shift:] = 0
        return augmented

In [ ]:
transform = TimeShift(always_apply=True, max_shift_second=4, sr=sr)
y_time_shifted = transform(y)
Audio(y_time_shifted, rate=sr)

In [ ]:
librosa.display.waveplot(y_time_shifted, sr=sr);

In [ ]:
melspec = librosa.power_to_db(librosa.feature.melspectrogram(y_time_shifted, sr=sr, n_mels=128))
librosa.display.specshow(melspec, sr=sr, x_axis="time", y_axis="mel")
plt.colorbar();

### VolumeControl

VolumeControlは音量を調節します。音の認識には音量そのものよりSNRが影響するという話を以前紹介したかと思いますが、音量を調節することでメルスペクトログラムにはごく僅かな変化が生じます。また、音量をサイン曲線、コサイン曲線などに合わせて調節する、などはメルスペクトログラムには大きな変化をもたらすため有用です。

VolumeControl controls the volume. I think I mentioned before that the SNR has more influence on sound perception than the volume itself, but adjusting the volume causes a very small change in the mel spectrogram. Adjusting the volume according to a sine curve, cosine curve, etc. is also useful because it causes a big change in the mel spectrogram.

In [ ]:
class VolumeControl(AudioTransform):
    def __init__(self, always_apply=False, p=0.5, db_limit=10, mode="uniform"):
        super().__init__(always_apply, p)

        assert mode in ["uniform", "fade", "fade", "cosine", "sine"], \
            "`mode` must be one of 'uniform', 'fade', 'cosine', 'sine'"

        self.db_limit= db_limit
        self.mode = mode

    def apply(self, y: np.ndarray, **params):
        db = np.random.uniform(-self.db_limit, self.db_limit)
        if self.mode == "uniform":
            db_translated = 10 ** (db / 20)
        elif self.mode == "fade":
            lin = np.arange(len(y))[::-1] / (len(y) - 1)
            db_translated = 10 ** (db * lin / 20)
        elif self.mode == "cosine":
            cosine = np.cos(np.arange(len(y)) / len(y) * np.pi * 2)
            db_translated = 10 ** (db * cosine / 20)
        else:
            sine = np.sin(np.arange(len(y)) / len(y) * np.pi * 2)
            db_translated = 10 ** (db * sine / 20)
        augmented = y * db_translated
        return augmented

In [ ]:
transform = VolumeControl(always_apply=True, mode="sine")
y_volume_controlled = transform(y)
Audio(y_volume_controlled, rate=sr)

In [ ]:
librosa.display.waveplot(y_volume_controlled, sr=sr);

In [ ]:
melspec = librosa.power_to_db(librosa.feature.melspectrogram(y_volume_controlled, sr=sr, n_mels=128))
librosa.display.specshow(melspec, sr=sr, x_axis="time", y_axis="mel")
plt.colorbar();

### 組み合わせて使う (Combination)

In [ ]:
transform = Compose([
  OneOf([
    GaussianNoiseSNR(min_snr=10),
    PinkNoiseSNR(min_snr=10)
  ]),
  PitchShift(max_steps=2, sr=sr),
  TimeStretch(),
  TimeShift(sr=sr),
  VolumeControl(mode="sine")
])
y_composed = transform(y)
Audio(y_composed, rate=sr)

In [ ]:
librosa.display.waveplot(y_composed, sr=sr);

In [ ]:
melspec = librosa.power_to_db(librosa.feature.melspectrogram(y_composed, sr=sr, n_mels=128))
librosa.display.specshow(melspec, sr=sr, x_axis="time", y_axis="mel")
plt.colorbar();

## [WIP] スペクトログラム/メルスペクトログラムに対するData Augmentation(Data Augmentation for waveform)

Work in progress...

## More to come. Stay tuned!